In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import random

import gym
from gym import spaces
import keras
from keras.layers import *
from keras.models import *
import numpy
import rl
import scipy.sparse
import skimage.io

import sys
print(sys.version)

%matplotlib inline

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


3.6.4 (default, Jan  2 2018, 20:24:15) 
[GCC 5.4.0 20160609]


In [ ]:
# input_neurons = 8
# inter_neurons = 128
# output_neurons = 8
# max_history = 16
# hidden_size = 32

In [3]:
import tensorflow as tf
from keras import backend as K

In [4]:
config = tf.ConfigProto(device_count = {'GPU': 1})
sess = tf.Session(config=config)
K.set_session(sess)

In [5]:
def sigmoid(x):
    return 1 / (1 + numpy.exp(-x))

In [6]:
def hebbian(history):
    return numpy.tanh(numpy.vectorize(lambda x: -1/5*(x-5))(history))

In [36]:
class SNN(gym.Env):
    
    def __init__(self, specification):
        self.specification = specification
        self.total_neurons = self.specification['input_neurons'] + self.specification['inter_neurons'] + self.specification['output_neurons']
        self.observation_space = gym.spaces.Box(-1, float('inf'), ((self.specification['inter_neurons'] + self.specification['output_neurons']) * (1 + 2 * (self.specification['input_neurons'] + self.specification['inter_neurons'])),), dtype=float)
        self.action_space = gym.spaces.Box(-1, 1, ((self.specification['inter_neurons'] + self.specification['output_neurons']) * (2 * (self.specification['input_neurons'] + self.specification['inter_neurons'])),), dtype=float)
#         self.observation_space = gym.spaces.Tuple((gym.spaces.Box(-1, 1, (self.specification['input_neurons'] + self.specification['inter_neurons'],)), gym.spaces.Box(-1, 1, (self.specification['input_neurons'] + self.specification['inter_neurons'],))))
        self.potential_matrix = numpy.zeros((self.total_neurons,))
        self.weight_matrix = numpy.zeros((self.specification['inter_neurons'] + self.specification['output_neurons'], self.specification['input_neurons'] + self.specification['inter_neurons']))
        self.weight_backup = self.weight_matrix.copy()
        self.weight_mask = numpy.ones_like(self.weight_matrix, dtype=numpy.uint8)
        self.weight_mask[-self.specification['output_neurons']:, :self.specification['input_neurons']] = 0
        numpy.fill_diagonal(self.weight_mask[:self.specification['inter_neurons'], -self.specification['inter_neurons']:], 0)
        self.history_matrix = numpy.zeros((self.specification['inter_neurons'] + self.specification['output_neurons'], 1 + 2 * (self.specification['input_neurons'] + self.specification['inter_neurons'])))
        self.random_seed = None
        self.next_input = None
        self.previous_reward = None
    
    def interconnect(self):
        self.weight_matrix = numpy.zeros_like(self.weight_matrix)
        probabilities = numpy.vectorize(lambda i, j: 1 / abs(i - j) if i != j else 0.)(*numpy.meshgrid(range(self.specification['inter_neurons']), range(self.specification['inter_neurons'])))
        mask = numpy.zeros((self.weight_matrix.shape[0], self.weight_matrix.shape[1]))
        mask[:-self.specification['output_neurons'], self.specification['input_neurons']:] = numpy.random.binomial(1, probabilities, (self.specification['inter_neurons'], self.specification['inter_neurons']))
        mask[:self.specification['inter_neurons'], :self.specification['input_neurons']] = numpy.eye(self.specification['inter_neurons'])[numpy.random.choice(self.specification['inter_neurons'], self.specification['input_neurons'])].swapaxes(0, 1)
        mask[self.specification['inter_neurons']:, self.specification['input_neurons']:] = numpy.eye(self.specification['inter_neurons'])[numpy.random.choice(self.specification['inter_neurons'], self.specification['output_neurons'])]
        rand1 = numpy.random.uniform(0, 1, (self.weight_matrix.shape[0], self.weight_matrix.shape[1]))
        rand1[:-self.specification['output_neurons'], self.specification['input_neurons']:] = rand1[:-self.specification['output_neurons'], self.specification['input_neurons']:] * 2 - 1
        rand2 = numpy.random.uniform(0, 1, (self.weight_matrix.shape[0], self.weight_matrix.shape[1]))
        self.weight_matrix = mask * rand1 * rand2 * self.weight_mask
        self.weight_backup = self.weight_matrix.copy()
#         self.sign_matrix = numpy.sign(self.weight_matrix)
    
    def load_weights(self, weights):
        self.weight_matrix[:, :] = weights
        self.sign_matrix[:, :] = numpy.sign(self.weight_matrix)
    
    def close(self):
        self.specification['environment'].close()
        
    def reset(self):
        self.next_input = self.specification['environment'].reset()
        self.potential_matrix = numpy.zeros_like(self.potential_matrix)
        self.weight_matrix = self.weight_backup
        self.history_matrix = numpy.zeros_like(self.history_matrix)
        return self.history_matrix.flatten()
    
    def render(self, mode='human'):
        return self.specification['environment'].render(mode=mode)
    
    def seed(self, seed):
        numpy.random.seed(seed)
        self.random_seed = seed
        return self.specification['environment'].seed(self.random_seed)
    
    def step(self, action):
#         self.next_input = np.eye(self.specification['input_neurons'])[np.random.choice(self.specification['input_neurons'], 1)]
        if self.specification['neuroplasticity']:
            action = action.reshape((self.specification['inter_neurons'] + self.specification['output_neurons'], 2 * (self.specification['input_neurons'] + self.specification['inter_neurons'])))
            action[self.specification['input_neurons'] + self.specification['inter_neurons']:] = numpy.round(sigmoid(action[self.specification['input_neurons'] + self.specification['inter_neurons']:]))
            action[:self.specification['input_neurons'] + self.specification['inter_neurons']] = numpy.tanh(action[:self.specification['input_neurons'] + self.specification['inter_neurons']])
            self.weight_matrix += self.specification['learning_rate'] * action[:, self.specification['input_neurons'] + self.specification['inter_neurons']:] * action[:, :self.specification['input_neurons'] + self.specification['inter_neurons']]
            self.weight_matrix = numpy.clip(numpy.multiply(self.weight_matrix, self.weight_mask), -1, 1)
#         state = numpy.zeros_like(self.history_matrix[self.neuron_idx, :])
#         reward = 0
#         terminal = False
        self.potential_matrix[:self.specification['input_neurons']] = numpy.add(self.potential_matrix[:self.specification['input_neurons']], self.next_input)
        firing_matrix = numpy.vectorize(lambda x: x >= 1)(self.potential_matrix)
        for i in range(self.specification['inter_neurons'] + self.specification['output_neurons']):
            pos = self.specification['input_neurons'] + i
            deltas = numpy.multiply(firing_matrix[:-self.specification['output_neurons']], self.weight_matrix[i])
            delta = numpy.sum(deltas)
            if self.specification['neuroplasticity']:
                self.history_matrix[i, self.specification['input_neurons'] + self.specification['inter_neurons']:] += 1
                self.history_matrix[i, self.specification['input_neurons'] + self.specification['inter_neurons']:-1] *= firing_matrix[:-self.specification['output_neurons']]
                self.history_matrix[i, -1] *= firing_matrix[pos]
                self.history_matrix[i, :self.specification['input_neurons'] + self.specification['inter_neurons']] = self.weight_matrix[i]
#                 self.history_matrix[i, self.weight_idx, self.specification['max_history'] - 1, :] = numpy.array([deltas[self.weight_idx], delta, self.potential_matrix[pos], firing_matrix[pos]])
            self.potential_matrix[pos] += delta
        self.potential_matrix = numpy.clip(numpy.multiply(self.potential_matrix, numpy.invert(firing_matrix)), -1, 1)
#         if self.specification['neuroplasticity']:
#             self.history_matrix = numpy.roll(self.history_matrix, 2, axis=1)
        state = self.history_matrix.flatten()
        self.next_input, reward, terminal, _ = self.specification['environment'].step(firing_matrix[-self.specification['output_neurons']:].astype(int))
        return state, reward, terminal, {}

In [8]:
class Test(gym.Env):
    
    def __init__(self):
        self.action_space = gym.spaces.Box(0, 1, (8,))
        self.observation_space = gym.spaces.Box(0, float('inf'), (8,))

    def close(self):
        pass

    def reset(self):
        self.state = numpy.zeros((8,))
        self.idx = 0
        return self.state

    def step(self, action):
        self.state = numpy.add(self.state, action)
        reward = ((self.state[1] - self.state[0]) * (self.state[2] - self.state[3])) * ((self.state[4] - self.state[5]) * (self.state[6] - self.state[7]))
        terminal = reward < 0
        self.idx += 1
        return numpy.ones(self.state.shape), reward, terminal, {}

In [9]:
class Test2(gym.Env):
    
    def __init__(self):
        self.action_space = gym.spaces.Discrete(1)
        self.observation_space = gym.spaces.Box(0, 1, (1,))
        self.state = None
        self.idx = None
        self.random_seed = None
    
    def seed(self, seed):
        self.random_seed = seed
        random.seed(seed)
        return seed
    
    def close(self):
        pass
    
    def reset(self):
        self.state = numpy.ones((1,))
        self.idx = 0
        return self.state
    
    def step(self, action):
        self.idx += 1
        terminal = self.idx == 10000
        if action[0] == 1:
            reward = 1.0
        else:
            reward = 0.0
        return self.state, reward, terminal, {}

In [13]:
class Test3(gym.Env):
    
    def __init__(self):
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(0, 1, (1,))
        self.state = None
        self.idx = None
        self.random_seed = None
        self.prev_action = None
    
    def seed(self, seed):
        self.random_seed = seed
        random.seed(seed)
        return seed
    
    def close(self):
        pass
    
    def reset(self):
        self.state = numpy.ones((1,))
        self.idx = 0
        self.prev_action = None
        return self.state
    
    def step(self, action):
        self.idx += 1
#         terminal = self.idx == 100000
        terminal = False
        reward = 0.
        if numpy.count_nonzero(action) == 1:
            if numpy.count_nonzero(self.prev_action) == 1:
                if numpy.argmax(action) != numpy.argmax(self.prev_action):
                    reward = 1.
        self.prev_action = action
        return self.state, reward, terminal, {}

In [11]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""

import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

class CartPoleEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5 # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation is still within bounds
        high = np.array([
            self.x_threshold * 2,
            np.finfo(np.float32).max,
            self.theta_threshold_radians * 2,
            np.finfo(np.float32).max])

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(-high, high, (8,))

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
#         assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state
        x, x_dot, theta, theta_dot = state
        force = self.force_mag * (2 * numpy.argmax(action) - 1) if numpy.count_nonzero(action) == 1 else 0
        costheta = math.cos(theta)
        sintheta = math.sin(theta)
        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta* temp) / (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
        xacc  = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        x  = x + self.tau * x_dot
        x_dot = x_dot + self.tau * xacc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * thetaacc
        self.state = (x,x_dot,theta,theta_dot)
        output = np.array([x < 0, x > 0, x_dot < 0, x_dot > 0, theta < 0, theta > 0, theta_dot < 0, theta_dot > 0]).astype(int)
        done =  x < -self.x_threshold or x > self.x_threshold or theta < -self.theta_threshold_radians or theta > self.theta_threshold_radians
        done = bool(done)

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn("You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.")
            self.steps_beyond_done += 1
            reward = 0.0

        return output, reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return numpy.zeros((8,))

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold*2
        scale = screen_width/world_width
        carty = 100 # TOP OF CART
        polewidth = 10.0
        polelen = scale * 1.0
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l,r,t,b = -cartwidth/2, cartwidth/2, cartheight/2, -cartheight/2
            axleoffset =cartheight/4.0
            cart = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l,r,t,b = -polewidth/2,polewidth/2,polelen-polewidth/2,-polewidth/2
            pole = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            pole.set_color(.8,.6,.4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth/2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5,.5,.8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0,carty), (screen_width,carty))
            self.track.set_color(0,0,0)
            self.viewer.add_geom(self.track)

        if self.state is None: return None

        x = self.state
        cartx = x[0]*scale+screen_width/2.0 # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array = mode=='rgb_array')

    def close(self):
        if self.viewer: self.viewer.close()

In [12]:
"""
http://incompleteideas.net/sutton/MountainCar/MountainCar1.cp
permalink: https://perma.cc/6Z2N-PFWC
"""

import math
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np

class MountainCarEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 30
    }

    def __init__(self):
        self.min_position = -1.2
        self.max_position = 0.6
        self.max_speed = 0.07
        self.goal_position = 0.5

        self.low = np.array([self.min_position, -self.max_speed])
        self.high = np.array([self.max_position, self.max_speed])

        self.viewer = None

        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(self.low, self.high)

        self.seed()
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        action = 2 * numpy.argmax(action) if numpy.count_nonzero(action) == 1 else 0
        assert self.action_space.contains(action), "%r (%s) invalid" % (action, type(action))

        position, velocity = self.state
        velocity += (action-1)*0.001 + math.cos(3*position)*(-0.0025)
        velocity = np.clip(velocity, -self.max_speed, self.max_speed)
        position += velocity
        position = np.clip(position, self.min_position, self.max_position)
        if (position==self.min_position and velocity<0): velocity = 0

        done = bool(position >= self.goal_position)
        reward = -1.0

        self.state = (position, velocity)
        
        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = np.array([self.np_random.uniform(low=-0.6, high=-0.4), 0])
        return np.array(self.state)

    def _height(self, xs):
        return np.sin(3 * xs)*.45+.55

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.max_position - self.min_position
        scale = screen_width/world_width
        carwidth=40
        carheight=20


        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            xs = np.linspace(self.min_position, self.max_position, 100)
            ys = self._height(xs)
            xys = list(zip((xs-self.min_position)*scale, ys*scale))

            self.track = rendering.make_polyline(xys)
            self.track.set_linewidth(4)
            self.viewer.add_geom(self.track)

            clearance = 10

            l,r,t,b = -carwidth/2, carwidth/2, carheight, 0
            car = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            car.add_attr(rendering.Transform(translation=(0, clearance)))
            self.cartrans = rendering.Transform()
            car.add_attr(self.cartrans)
            self.viewer.add_geom(car)
            frontwheel = rendering.make_circle(carheight/2.5)
            frontwheel.set_color(.5, .5, .5)
            frontwheel.add_attr(rendering.Transform(translation=(carwidth/4,clearance)))
            frontwheel.add_attr(self.cartrans)
            self.viewer.add_geom(frontwheel)
            backwheel = rendering.make_circle(carheight/2.5)
            backwheel.add_attr(rendering.Transform(translation=(-carwidth/4,clearance)))
            backwheel.add_attr(self.cartrans)
            backwheel.set_color(.5, .5, .5)
            self.viewer.add_geom(backwheel)
            flagx = (self.goal_position-self.min_position)*scale
            flagy1 = self._height(self.goal_position)*scale
            flagy2 = flagy1 + 50
            flagpole = rendering.Line((flagx, flagy1), (flagx, flagy2))
            self.viewer.add_geom(flagpole)
            flag = rendering.FilledPolygon([(flagx, flagy2), (flagx, flagy2-10), (flagx+25, flagy2-5)])
            flag.set_color(.8,.8,0)
            self.viewer.add_geom(flag)

        pos = self.state[0]
        self.cartrans.set_translation((pos-self.min_position)*scale, self._height(pos)*scale)
        self.cartrans.set_rotation(math.cos(3 * pos))

        return self.viewer.render(return_rgb_array = mode=='rgb_array')

    def close(self):
        if self.viewer: self.viewer.close()

In [ ]:
env = SNN({'environment': Test2(), 'input_neurons': 1, 'inter_neurons': 10, 'output_neurons': 1,  'neuroplasticity': True, 'learning_rate': 0.1})
numpy.random.seed(0)
env.seed(0)
env.interconnect()
env.reset()

In [ ]:
skimage.io.imshow(env.weight_matrix)

In [ ]:
action = numpy.zeros(11 * (2 * 11))

In [ ]:
for i in range(1):
    stuff = env.step(action)

In [ ]:
skimage.io.imshow(env.potential_matrix.reshape((3, 4)))

In [ ]:
gym.undo_logger_setup()

In [ ]:
input_neurons = 1
inter_neurons = 10
output_neurons = 1

In [ ]:
neurons = input_neurons + inter_neurons + output_neurons

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

from rl.agents import NAFAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess
from rl.core import Processor

In [ ]:
# Get the environment and extract the number of actions.
env = SNN({
    'environment': Test2(),
    'input_neurons': input_neurons,
    'inter_neurons': inter_neurons,
    'output_neurons': output_neurons,
    'neuroplasticity': True,
    'learning_rate': 0.1})
numpy.random.seed(0)
env.seed(0)
assert len(env.action_space.shape) == 1
nb_actions = env.action_space.shape[0]

In [ ]:
(1,) + env.observation_space.shape

In [ ]:
nb_actions

In [ ]:
# Build all necessary models: V, mu, and L networks.
V_model = Sequential()
V_model.add(Reshape(input_shape=(1,) + env.observation_space.shape, target_shape=(inter_neurons + output_neurons, 1 + 2 * (input_neurons + inter_neurons))))
V_model.add(TimeDistributed(Dense(neurons, activation='relu')))
V_model.add(TimeDistributed(Dense(neurons, activation='relu')))
V_model.add(TimeDistributed(Dense(neurons, activation='relu')))
V_model.add(Flatten())
V_model.add(Dense(1, activation='linear'))
print(V_model.summary())

mu_model = Sequential()
mu_model.add(Reshape(input_shape=(1,) + env.observation_space.shape, target_shape=(inter_neurons + output_neurons, 1 + 2 * (input_neurons + inter_neurons))))
mu_model.add(TimeDistributed(Dense(neurons, activation='relu')))
mu_model.add(TimeDistributed(Dense(neurons, activation='relu')))
mu_model.add(TimeDistributed(Dense(neurons, activation='relu')))
mu_model.add(TimeDistributed(Dense(2 * (input_neurons + inter_neurons), activation='tanh')))
mu_model.add(Flatten())
print(mu_model.summary())

action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
x = Concatenate()([action_input, Flatten()(observation_input)])
x = Dense(2 * neurons, activation='relu')(x)
x = Dense(2 * neurons, activation='relu')(x)
x = Dense(2 * neurons, activation='relu')(x)
x = Dense(((nb_actions * nb_actions + nb_actions) // 2))(x)
x = Activation('linear')(x)
L_model = Model(inputs=[action_input, observation_input], outputs=x)
print(L_model.summary())

In [ ]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(theta=.15, mu=0., sigma=.3, size=nb_actions)
agent = NAFAgent(nb_actions=nb_actions, V_model=V_model, L_model=L_model, mu_model=mu_model,
                 memory=memory, nb_steps_warmup=100, random_process=random_process,
                 gamma=.99, target_model_update=1e-3, processor=None)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

In [ ]:
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
agent.fit(env, nb_steps=50000, visualize=False, verbose=2, nb_max_episode_steps=200)

In [ ]:
env.weight_matrix

In [ ]:
# After training is done, we save the final weights.
# agent.save_weights('cdqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
agent.test(env, nb_episodes=10, visualize=False, nb_max_episode_steps=200)

In [15]:
input_neurons = 1
inter_neurons = 10
output_neurons = 2
neurons = input_neurons + inter_neurons + output_neurons

In [16]:
import numpy as np
import gym

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

In [17]:
gym.undo_logger_setup()

/home/santient/.local/lib/python3.6/site-packages/gym/__init__.py:15: UserWarning: gym.undo_logger_setup is deprecated. gym no longer modifies the global logging configuration
  warnings.warn("gym.undo_logger_setup is deprecated. gym no longer modifies the global logging configuration")


In [43]:
# Get the environment and extract the number of actions.
env = SNN({
    'environment': Test3(),
    'input_neurons': input_neurons,
    'inter_neurons': inter_neurons,
    'output_neurons': output_neurons,
    'neuroplasticity': True,
    'learning_rate': 0.1})
np.random.seed(123)
env.seed(123)
env.interconnect()
assert len(env.action_space.shape) == 1
nb_actions = env.action_space.shape[0]

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [44]:
# # Next, we build a very simple model.
# actor = Sequential()
# actor.add(Reshape(input_shape=(1,) + env.observation_space.shape, target_shape=(inter_neurons + output_neurons, 1 + 2 * (input_neurons + inter_neurons))))
# actor.add(TimeDistributed(Dense(neurons, activation='relu')))
# actor.add(TimeDistributed(Dense(neurons, activation='relu')))
# actor.add(TimeDistributed(Dense(neurons, activation='relu')))
# actor.add(TimeDistributed(Dense(2 * (input_neurons + inter_neurons), activation='linear')))
# actor.add(Flatten())
# print(actor.summary())

# action_input = Input(shape=(nb_actions,), name='action_input')
# observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
# reshaped_action = Reshape((inter_neurons + output_neurons, 2 * (input_neurons + inter_neurons)))(action_input)
# reshaped_observation = Reshape((inter_neurons + output_neurons, 1 + 2 * (input_neurons + inter_neurons)))(observation_input)
# x = Concatenate()([reshaped_action, reshaped_observation])
# x = TimeDistributed(Dense(2 * neurons, activation='relu'))(x)
# x = TimeDistributed(Dense(2 * neurons, activation='relu'))(x)
# x = TimeDistributed(Dense(2 * neurons, activation='relu'))(x)
# x = Flatten()(x)
# x = Dense(1, activation='linear')(x)
# critic = Model(inputs=[action_input, observation_input], outputs=x)
# print(critic.summary())

In [45]:
# Next, we build a very simple model.
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(neurons * (inter_neurons + output_neurons), activation='relu'))
actor.add(Dense(neurons * (inter_neurons + output_neurons), activation='relu'))
actor.add(Dense(neurons * (inter_neurons + output_neurons), activation='relu'))
actor.add(Dense(2 * (inter_neurons + output_neurons) * (input_neurons + inter_neurons), activation='tanh'))
print(actor.summary())

action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = Concatenate()([action_input, flattened_observation])
x = Dense(2 * (inter_neurons + output_neurons) * neurons, activation='relu')(x)
x = Dense(2 * (inter_neurons + output_neurons) * neurons, activation='relu')(x)
x = Dense(2 * (inter_neurons + output_neurons) * neurons, activation='relu')(x)
x = Dense(1, activation='linear')(x)
critic = Model(inputs=[action_input, observation_input], outputs=x)
print(critic.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_15 (Flatten)         (None, 276)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 156)               43212     
_________________________________________________________________
dense_58 (Dense)             (None, 156)               24492     
_________________________________________________________________
dense_59 (Dense)             (None, 156)               24492     
_________________________________________________________________
dense_60 (Dense)             (None, 264)               41448     
Total params: 133,644
Trainable params: 133,644
Non-trainable params: 0
_________________________________________________________________
None
__________________________________________________________________________________________________
Layer (type)                    

In [46]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=.15, mu=0., sigma=.3)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

In [47]:
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
agent.fit(env, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=100)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 218s 22ms/step - reward: 0.1896
100 episodes - episode_reward: 18.960 [0.000, 91.000] - loss: 4.707 - mean_absolute_error: 0.911 - mean_q: 22.055

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 219s 22ms/step - reward: 0.2669
100 episodes - episode_reward: 26.690 [0.000, 79.000] - loss: 514.163 - mean_absolute_error: 7.564 - mean_q: 262.760

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 218s 22ms/step - reward: 0.0188
100 episodes - episode_reward: 1.880 [0.000, 12.000] - loss: 27801.027 - mean_absolute_error: 49.593 - mean_q: 2055.306

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 219s 22ms/step - reward: 0.0000e+00
100 episodes - episode_reward: 0.000 [0.000, 0.000] - loss: 1086415.000 - mean_absolute_error: 265.231 - mean_q: 13486.728

Interval 5 (40000 steps performed)
10

In [ ]:
# After training is done, we save the final weights.
# agent.save_weights('ddpg_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
agent.test(env, nb_episodes=5, visualize=False, nb_max_episode_steps=1000)

In [ ]:
skimage.io.imshow(env.weight_matrix)